# DS Automation Assignment

Using our prepared churn data from week 2:
- use pycaret to find an ML algorithm that performs best on the data
    - Choose a metric you think is best to use for finding the best model; by default, it is accuracy but it could be AUC, precision, recall, etc. The week 3 FTE has some information on these different metrics.
- save the model to disk
- create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe
    - your Python file/function should print out the predictions for new data (new_churn_data.csv)
    - the true values for the new data are [1, 0, 0, 1, 0] if you're interested
- test your Python module and function with the new data, new_churn_data.csv
- write a short summary of the process and results at the end of this notebook
- upload this Jupyter Notebook and Python file to a Github repository, and turn in a link to the repository in the week 5 assignment dropbox

*Optional* challenges:
- return the probability of churn for each new prediction, and the percentile where that prediction is in the distribution of probability predictions from the training dataset (e.g. a high probability of churn like 0.78 might be at the 90th percentile)
- use other autoML packages, such as TPOT, H2O, MLBox, etc, and compare performance and features with pycaret
- create a class in your Python module to hold the functions that you created
- accept user input to specify a file using a tool such as Python's `input()` function, the `click` package for command-line arguments, or a GUI
- Use the unmodified churn data (new_unmodified_churn_data.csv) in your Python script. This will require adding the same preprocessing steps from week 2 since this data is like the original unmodified dataset from week 1.

In [1]:
# import libraries
import pandas as pd
import os as os
from pycaret.classification import setup, compare_models, predict_model, save_model, load_model

In [2]:
# load data
df = pd.read_csv('./data/prepared_churn_data.csv', index_col='customerID')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tenure             7043 non-null   float64
 1   PhoneService       7043 non-null   int64  
 2   Contract           7043 non-null   int64  
 3   PaymentMethod      7043 non-null   int64  
 4   MonthlyCharges     7043 non-null   float64
 5   TotalCharges       7043 non-null   float64
 6   Churn              7043 non-null   int64  
 7   charge_per_tenure  7043 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 495.2+ KB


In [3]:
# Setup pycaret, ignoring customerID
automl = setup(df, target='Churn', preprocess=False, ignore_features=['customerID'])

,Description,Value
0,Session id,5589
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(7043, 8)"
4,Transformed data shape,"(7043, 8)"
5,Transformed train set shape,"(4930, 8)"
6,Transformed test set shape,"(2113, 8)"
7,Ignore features,1
8,Numeric features,7


In [4]:
# have pycaret compare models
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7929,0.8309,0.5114,0.6367,0.5666,0.4328,0.4376,0.2430
gbc,Gradient Boosting Classifier,0.7923,0.8358,0.4900,0.6426,0.5558,0.4235,0.4303,0.0520
ridge,Ridge Classifier,0.7897,0.8179,0.4381,0.6539,0.5239,0.3959,0.4093,0.0040
lda,Linear Discriminant Analysis,0.7890,0.8179,0.4900,0.6307,0.5509,0.4162,0.4221,0.0040
lightgbm,Light Gradient Boosting Machine,0.7864,0.8237,0.5167,0.6161,0.5614,0.4219,0.4251,0.3160
ada,Ada Boost Classifier,0.7828,0.8314,0.4755,0.6174,0.5366,0.3980,0.4041,0.0190
rf,Random Forest Classifier,0.7777,0.7995,0.4686,0.6048,0.5277,0.3855,0.3910,0.0480
svm,SVM - Linear Kernel,0.7698,0.7495,0.4259,0.6014,0.4844,0.3463,0.3606,0.0060
knn,K Neighbors Classifier,0.7641,0.7514,0.4265,0.5736,0.4886,0.3399,0.3464,0.0670
et,Extra Trees Classifier,0.7641,0.7828,0.4793,0.5655,0.5185,0.3638,0.3662,0.0360


In [6]:
# save best model using the class name as the filename... this will make subsequent trials touch-free
model_name = best_model.__class__.__name__
save_model(best_model, model_name)

Transformation Pipeline and Model Successfully Saved


In [7]:
# load the pickled model
loaded_model =load_model(model_name)
loaded_model

Transformation Pipeline and Model Successfully Loaded


In [8]:
# test the loaded model by making a prediction on unseen data
unseen_data = df.copy()
unseen_data = unseen_data.drop('Churn', axis=1)
predict_model(loaded_model, data=unseen_data.iloc[-2:-1])

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,charge_per_tenure,prediction_label,prediction_score
customerID,,,,,,,,,
8361-LTMKD,4.0,1,0,2,74.400002,306.600006,76.650002,1,0.5695


In [9]:
# vefify the prediction
df.iloc[-2:-1]

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,charge_per_tenure
customerID,,,,,,,,
8361-LTMKD,4.0,1,0,2,74.4,306.6,1,76.65


In [11]:
# load libraries from python file
from predict_churn import make_prediction

# load unseen data from a prepared file
new_data = pd.read_csv('./data/new_churn_data.csv', index_col='customerID')

# make the prediction and print the result
print( make_prediction(new_data, model_name) )

Transformation Pipeline and Model Successfully Loaded


            Label   Score
customerID               
9305-CKSKC      1  0.5120
1452-KNGVK      1  0.5956
6723-OKKJM      0  0.9388
7832-POPKP      0  0.8249
6348-TACGU      1  0.7029


# Summary

In this exercise, we are using pycaret to automate machine learning tasks. 

After loading the data, I initialized PyCaret with the 'setup' function.  I passed it the training and test data, and the target variable.  PyCaret has functions for preprocessing data, but I'm opting not to use them because we've already preprocessed the data in week 2.  I also invoke the ignore_features parameter so that CustomerID was not evaluated in the models.  In past exercises, I dropped that column before training my models... it's nice to have the ignore option.

I then have PyCaret evaluate and compare multiple models, sorting by accuracy (the default).




Write a short summary of the process and results here.